### Portfolio Optimization - Wasserstein Ball and Reinforcement Learning

* [1. Import Packages](#1)
* [2. Stock Simulation Function](#2)  
* [3. ANN for mapping State to action](#3)      
* [4. Function toretun portfolio value and Risk Measure](#4)    
* [5. Training](#5)

<a id='1'></a>
### 1. Import Packages

In [ ]:
#!/usr/bin/env python
# coding: utf-8
import numpy as np
import numpy.matlib
from scipy.stats import norm

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from scipy import stats
import seaborn as sns

<a id='2'></a>
### 2. Stock Simulation Function

In [ ]:
# %%
def SimPath(Ndt, S0, mu, sigma, T, Nsims):
    
    dt = T/Ndt
    
    S = np.zeros((Nsims,Ndt+1))
    S[:,0] = S0
    
    for i in range(Ndt):
        
        dW = np.sqrt(dt) * np.random.randn(Nsims)
        S[:,i+1] = S[:,i] * np.exp((mu-0.5*sigma**2)*dt + sigma*dW)
        
    t = np.linspace(0,T,Ndt+1)
    return t, S
        


# %%
t, S = SimPath(252,1, 0.1, 0.2, 5, 1000)

# %%
Ndt=20

S0=1

T=5

Nsims=50


plt.plot(t, S.T,linewidth=0.1)
plt.plot(t,np.quantile(S,[0.1, 0.5, 0.9],axis=0).T,color='k',linewidth=2)
plt.xlabel("Time")
plt.ylabel("Asset Price $S_t$")
plt.xlim([0,5])
plt.show()

<a id='3'></a>
### 3. ANN for mapping state and weights
In this function we pass the prices of the instruments and get the weights across time

In [ ]:
# %%
class MyNet(nn.Module):
    
    def __init__(self, n ):
        super(MyNet, self).__init__()
        
        # 3 input layer (t,S), 1 output channel, 3 hidden layers with n units each
        self.f_in_to_h1 = nn.Linear( 4 , n)
        self.f_h1_to_h2 = nn.Linear(n, n)
        self.f_h2_to_out = nn.Linear(n, 3)
        self.myReLU = nn.ReLU()

    def forward(self, x):
        
        # input into 1st hidden layer
        h1 = self.myReLU(self.f_in_to_h1(x) )
        
        # 1st hidden to 2nd hidden layer
        h2 = self.myReLU(self.f_h1_to_h2(h1))
            
        # 2nd hidden layer to output layer
        y = self.f_h2_to_out(h2)
        
        return y

<a id='3'></a>
### 3. Helper functions - Risk Measure and Wassterestein function
This function fetches the Return, risk measure and portfolio path for each path

In [ ]:
def GetRiskMeasure(Return, Type = "TVaR"):
    #TVaR    
    if (Type == "TVaR"):
        percentile = 5
        k = 1 + round(.01 * float(percentile) * (Return[1,:].numel() - 1))
        RiskMeasure =  -1*Return.kthvalue(k).values
        #RiskMeasure = torch.tensor(-1* np.percentile(Return.detach().numpy(), 5,axis=1), dtype=torch.float, requires_grad=True)
    #Variance
    elif (Type == "Variance"):  
        RiskMeasure = torch.var(Return)   
    return RiskMeasure        

In [ ]:
def getWassDistance(TotalVal_T, IndexVal_T):
    wd = stats.wasserstein_distance(TotalVal_T.detach(),IndexVal_T.detach())
    wass_dist = torch.tensor(wd, dtype=torch.float, requires_grad=True)
    
    ## Other minimizations
    #wd = stats.wasserstein_distance(wt[:,-1,0].detach(),Delta_t[:,-1,0].detach())
    #wass_dist = wt-Delta_t
    #wass_dist = wt[:,-1,:] - Delta_t[:,-1,:]
    #wass_dist = (TotalVal_T - IndexVal_T)**2
    return wass_dist  

In [ ]:
t, S1 = SimPath(Ndt, 1, .05, .1, T, Nsims)
t, S2 = SimPath(Ndt, 1, .06, .12, T, Nsims)
t, S3 = SimPath(Ndt, 1, .06, .12, T, Nsims)

t

x=np.zeros((Nsims,Ndt+1,4))
x.shape 

x[:,:,0] = np.matlib.repmat(t,Nsims, 1)

x[:,:,1] = S1

x[:,:,2] = S2

x[:,:,3] = S3

xt = torch.tensor(x, dtype=torch.float)

wt= torch.tensor(np.zeros((Nsims,Ndt+1,3)), dtype=torch.float)

wt[:,:-1,:].shape

In [ ]:
wt[:,1:,:] = net(xt[:,:-1,:])
wt.shape

#Compute wt
#Set the t0 of the portfolio same as index
wt[:,0,:] = Delta_t[:,0,:]

S1.shape

wt[:,:,1].shape

wt 

S1[:,:-1].shape

Val1= wt[:,:,0] * torch.tensor(S1, dtype=torch.float)
Val2= wt[:,:,1] * torch.tensor(S2, dtype=torch.float)
Val3= wt[:,:,2] * torch.tensor(S3, dtype=torch.float)
TotalVal = Val1 + Val2 + Val3

TotalVal.shape

In [ ]:
#Weights at time t 
#TotalVal[:, -1]

x2=np.zeros((Nsims,Ndt-1))
Return = torch.tensor(x2, dtype=torch.float)
Return.shape

for i in range(1,Ndt-1):
    Return[:,i] = TotalVal[:,i+1] - TotalVal[:,i]
        

wt.shape

#output for one scenario
#wt[0,:,:]

#Weight at T for all scenarios 
wt[:,-1,:].shape

In [ ]:
Delta_t = torch.tensor(np.zeros((Nsims,Ndt+1,3)), dtype=torch.float)
Delta_t[:,:,0] = .50
Delta_t[:,:,1] = .25
Delta_t[:,:,2] = .25

Val1_index= Delta_t[:,:,0] * torch.tensor(S1, dtype=torch.float, requires_grad=True)
Val2_index= Delta_t[:,:,1] * torch.tensor(S2, dtype=torch.float, requires_grad=True)
Val3_index= Delta_t[:,:,2] * torch.tensor(S2, dtype=torch.float, requires_grad=True)
TotalVal_index = Val1_index + Val2_index + Val3_index
#TotalVal_index[:, -1]
Delta_t[:,-1,:].shape
#np.array(TotalVal_index.detach())[3]

In [ ]:
Return = TotalVal[:,1:] - TotalVal[:,:-1] 

#Return at time T across all the scenarios
Return.shape

In [ ]:
#np.array(TotalVal.detach()).mean(axis =0)

#np.percentile(Return.detach().numpy(), 5, axis=1)

RiskMeasure = torch.tensor(np.percentile(Return.detach().numpy(), 5,axis=1), dtype=torch.float, requires_grad=True)
RiskMeasure

In [ ]:
stats.wasserstein_distance(TotalVal_T.detach(),IndexVal_T.detach())

#TotalVal_T

#stats.wasserstein_distance(1,2)
#TotalVal_T

Return.shape

Return.kthvalue(k).values

percentile = 5
k = 1 + round(.01 * float(percentile) * (Return[1,:].numel() - 1))
Return.kthvalue(k).values

In [ ]:
Return[1,:].numel()

round(.01 * float(.05) * (Return.numel() - 1))

wt, Return, TVaR, TotalVal, TotalVal_T, TotalVal_index,IndexVal_T, wass_dist = SimTVaR(net, Ndt, T, Nsims, True)

In [ ]:
#Other minimizations
#wd = stats.wasserstein_distance(wt[:,-1,0].detach(),Delta_t[:,-1,0].detach())
#wass_dist = wt-Delta_t
#wass_dist = wt[:,-1,:] - Delta_t[:,-1,:]
#wass_dist = (TotalVal_T - IndexVal_T)**2

#Return at time T across all the scenarios     



#TVaR   
percentile = 5
k = 1 + round(.01 * float(percentile) * (Return[1,:].numel() - 1))
RiskMeasure =  -1*Return.kthvalue(k).values
#RiskMeasure = torch.tensor(-1* np.percentile(Return.detach().numpy(), 5,axis=1), dtype=torch.float, requires_grad=True)

<a id='3'></a>
### 3. Function to get the Return, risk measure and portfolio weights
This function fetches the Return, risk measure and portfolio path for each path

In [ ]:
Ndt = 100
S1_T0 = 1
S2_T0 = 100
S3_T0 = 10
Nsims = 50

def SimTVaR(net, Ndt, T, Nsims, ReturnTensor):
    
    #The function is designed for 3 stocks. Can be customized for any number of stocks
    num_asset = 3   
    
    t, S1 = SimPath(Ndt, S1_T0, .05, .1, T, Nsims)
    t, S2 = SimPath(Ndt, S2_T0, .06, .12, T, Nsims)
    t, S3 = SimPath(Ndt, S3_T0, .06, .12, T, Nsims)
    
    #Index Delta and return
    Delta_t = torch.tensor(np.zeros((Nsims,Ndt+1,num_asset)), dtype=torch.float)
    Delta_t[:,:,0] = .50
    Delta_t[:,:,1] = .25
    Delta_t[:,:,2] = .25

    Val1_index= Delta_t[:,:,0] * torch.tensor(S1, dtype=torch.float, requires_grad=True)
    Val2_index= Delta_t[:,:,1] * torch.tensor(S2, dtype=torch.float, requires_grad=True)
    Val3_index= Delta_t[:,:,2] * torch.tensor(S3, dtype=torch.float, requires_grad=True)
    TotalVal_index = Val1_index + Val2_index + Val3_index
    IndexVal_T=TotalVal_index[:, -1]
    
    #Getting the weights for the portfolio
    x=np.zeros((Nsims,Ndt+1,num_asset+1))   
    x[:,:,1] = S1
    x[:,:,2] = S2
    x[:,:,3] = S3
    xt = torch.tensor(x, dtype=torch.float)
    wt= Delta_t
    wt[:,1:,:] = net(xt[:,:-1,:])
    #wt[:,0,:] = Delta_t[:,0,:]
    Val1= wt[:,:,0] * torch.tensor(S1, dtype=torch.float)
    Val2= wt[:,:,1] * torch.tensor(S2, dtype=torch.float)
    Val3= wt[:,:,2] * torch.tensor(S3, dtype=torch.float)
    TotalVal = Val1 + Val2 + Val3
    TotalVal_T = TotalVal[:,-1]   
    #Compute the return between two consecutive days
    Return = TotalVal[:,1:] - TotalVal[:,:-1]  
    
    
    # Compute the risk measure and wasstertein Distance for the computation
    wass_dist = getWassDistance(TotalVal_T, IndexVal_T)
    RiskMeasure = GetRiskMeasure(Return, "TVaR")    
    
    return wt, Return, RiskMeasure, TotalVal, TotalVal_index, TotalVal_T, IndexVal_T, wass_dist

<a id='4'></a>
### 4. Training Function

In [ ]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print('Using device:', device)
#net.to(device)

#print(net)

#Create object of the class
net = MyNet(80)

# create  optimizer
optimizer = optim.Adam(net.parameters())


Nepochs = 10000
loss_hist = []

for epoch in range(Nepochs):  # loop over the dataset multiple times


    # zero the parameter gradients
    optimizer.zero_grad()


    #hedge_payoff, true_payoff, S = SimHedge(net, Ndt, S0, mu, sigma, T, Nsims, True)
    wt, Return, RiskMeasure, TotalVal, TotalVal_index, TotalVal_T, IndexVal_T, wass_dist = SimTVaR(net, Ndt, T, Nsims, True)
    
    #print (S )
    #error = wass_dist + TVaR
    #print (error )
    regularization_param = 1
    loss = regularization_param*torch.sum(wass_dist) + torch.sum(RiskMeasure)
    #loss = torch.sum(-TVaR)
    #print (wt) 
    loss.backward()   
    
    #print (wt, Return, TVaR ) 
    # optimize
    optimizer.step()
    
    # store running loss
    loss_hist.append(  loss.item() )
    
    # plot output every 50 iterations
    if( (epoch % 500 == 0) and (epoch>1) ):
        print(epoch,end=" ")
        #print(loss.item())
        print("Wass Dist:", regularization_param*torch.sum(wass_dist**2))
        print("Risk Measure:", torch.sum(RiskMeasure))
        sns.distplot(np.array(IndexVal_T.detach()), hist=True, kde=True, label='Index')
        sns.distplot(np.array(TotalVal_T.detach()), hist=True, kde=True, label='Portfolio')
        plt.figure(figsize=(5,3))
        plt.plot(loss_hist)
        plt.show()
        
        plt.figure(figsize=(5,3))
        plt.plot(np.array(TotalVal_index.detach())[0], label='Index')
        plt.plot(np.array(TotalVal.detach())[0], label='Portfolio')
        plt.legend()
        plt.show()

print('Finished Training')

### Rough Code for testing